<a href="https://colab.research.google.com/github/JALVARADORUIZ/Magister_BigData_UB/blob/main/grupo14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importación de Librerias**

In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import nltk
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

## **Carga de Data**

In [ ]:
reviews_file = "https://raw.githubusercontent.com/albertojulian/master-ub/main/reviews.txt"
labels_file = "https://raw.githubusercontent.com/albertojulian/master-ub/main/labels.txt"

In [ ]:

df_reviews = pd.read_csv(reviews_file,header=None, names=['review'])
df_labels = pd.read_csv(labels_file, header=None, names=['sentiment'])

df = pd.concat([df_reviews,df_labels],axis=1)

In [ ]:
df.head(5)

,review,sentiment
0,bromwell high is a cartoon comedy . it ran at ...,positive
1,story of a man who has unnatural feelings for ...,negative
2,homelessness or houselessness as george carli...,positive
3,airport starts as a brand new luxury pla...,negative
4,brilliant over acting by lesley ann warren . ...,positive


In [ ]:
df['sentiment'].value_counts()

positive    12500
negative    12500
Name: sentiment, dtype: int64

## **Preprocesamiento**

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


- Eliminar signos que no sean alfanuméricos.

In [ ]:
def preprocess_string(s):
    # Eliminar todos los caracteres que no sean palabras (todo excepto números y letras)
    s = re.sub(r"[^\w\s]", '', s)
    # Reemplazar todas las series de espacios en blanco sin espacio
    s = re.sub(r"\s+", '', s)
    # reemplazar dígitos sin espacio
    s = re.sub(r"\d", '', s)

    return s

- Tokenizar cada revisión (convertirla en una lista de palabras).
- Generar la lista de palabras distintas de todo el dataset y ordenar por número de veces que se presenta en el dataset, de forma que se puedan seleccionar las “n” más frecuentes.
- Asociar cada palabra de la lista con un número identificador.

In [ ]:
def tockenize(x,y):
    word_list = []
    #utilizamos stopwords
    stop_words = set(stopwords.words('english')) 
    for sent in x:
        
        for word in sent.split():
            word = preprocess_string(word)
            if word not in stop_words and word != '':
                word_list.append(word)
  
    corpus = Counter(word_list)
    # clasificación en base a las palabras más comunes
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]

    # creamos un diccionario
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_)}
    
    # tokenizamos y Generar la lista de palabras distintas de todo el dataset 
    final_list = []
    for sent in x:
            final_list.append([onehot_dict[preprocess_string(word)] for word in sent.split() 
                                     if preprocess_string(word) in onehot_dict.keys()])
            
    encoded = [1 if label =='positive' else 0 for label in y]  

    return np.array(final_list), np.array(encoded),onehot_dict

In [ ]:
reviews, sentiments, vocab = tockenize(df.review, df.sentiment)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
print("Tamaño de vocabulario:", len(vocab))

Tamaño de vocabulario: 1000


In [ ]:
len(reviews),len(sentiments)

(25000, 25000)

- Truncar las revisiones a 160 palabras y rellenar con “0” las revisiones que tengan menos palabras.

In [ ]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len), dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [ ]:
X = padding_(reviews, 160)
y = sentiments

In [ ]:
len(X)

25000

- Dividir los datasets de revisiones y etiquetas en 80% para entrenamiento, 10% para validación y 10% para pruebas.

In [ ]:
# 80% de nuestra data para train
x_train, y_train = X[:20000], y[:20000]
len(x_train),len(y_train)

(20000, 20000)

In [ ]:
# Dejamos 10% para validacion
x_val, y_val = X[20000:22500], y[20000:22500]
len(x_val),len(y_val)

(2500, 2500)

In [ ]:
# Dejamos el 10% para test
x_test,y_test = X[22500:25000], y[22500:25000]
len(x_test),len(y_test)

(2500, 2500)

## **Carga del fichero de palabras vectorizadas GloVe de 100 dimensiones (se puede eliminar el resto de ficheros)** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!cp /content/glove.6B.zip /content/drive/MyDrive/data/glove.6B.zip
#!unzip /content/drive/MyDrive/data/glove.6B.zip -d /content/drive/MyDrive/data/

--2022-07-30 22:31:59--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-07-30 22:31:59--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-07-30 22:32:00--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
embeddings_index = {}
f = open('/content/drive/MyDrive/data/glove.6B.100d.txt')

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
  if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

NameError: ignored